In [1]:
from tqdm import tqdm
from langchain_community.vectorstores import FAISS
import pickle
import pandas as pd
import numpy as np
from langchain.schema import Document
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.document_loaders import DataFrameLoader
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

In [2]:
class CFG:
    # store="프랭크버거"
    output_path = "/home/user09/beaver/data/db"
    save_path = ""
    embedding_model="BAAI/bge-m3"
    retriever_k=5
    retriever_bert_weight=0.7
    version='11'
    seed=42
    
# CFG.save_path = CFG.output_path + CFG.store + "_temp.csv"

In [3]:
store = "홍콩반점"

df = pd.read_excel(f'/home/user09/beaver/data/shared_files/dataset/dataset_v{CFG.version}.xlsx')

In [4]:
# Extracting information from the "Details" column to create documents
docs = []
for _, row in df.iterrows():
    details = row['Details']
    # metadata = {}
    
    # # Parse key-value pairs in the "Details" column
    # for detail in details.split('\n'):
    #     if ':' in detail:
    #         key, value = detail.split(':', 1)
            # metadata[key.strip()] = value.strip()
    
    docs.append(Document(page_content=details))#, metadata=metadata))

# Embeddings and vector store setup
encode_kwargs = {'normalize_embeddings': True}
model_kwargs = {'device': 'cpu'}

hf = HuggingFaceBgeEmbeddings(
    model_name=CFG.embedding_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

db = FAISS.from_documents(docs, hf)

# Save the FAISS vector store and documents as pickle
db.save_local(f"{CFG.output_path}/{store}_faiss{CFG.version}")

with open(f"{CFG.output_path}/{store}_docs{CFG.version}.pkl", "wb") as f:
    pickle.dump(docs, f)

# Load the FAISS vector store
db = FAISS.load_local(f"/home/user09/beaver/data/db/{store}_faiss{CFG.version}", hf, allow_dangerous_deserialization=True)

# Create retrievers
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": CFG.retriever_k})
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = CFG.retriever_k

ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever],
    weights=[CFG.retriever_bert_weight, 1 - CFG.retriever_bert_weight]
)


/home/user09/venv/beaver/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
# Example input and retrieving relevant documents
input_query = "영업시간이 어떻게 돼"
retrieved_docs = ensemble_retriever.get_relevant_documents(input_query)

# Print out the retrieved documents
for i, doc in enumerate(retrieved_docs, 1):
    print(f"Document {i}:\nContent: {doc.page_content}\nMetadata: {doc.metadata}\n")

Document 1:
Content: 영업 시간: 11:30 - 21:3020:30 라스트오더
Metadata: {}

Document 2:
Content: 매장 전화번호: 02-555-8883
Metadata: {}

Document 3:
Content: 매장 위치: 서울 강남구 테헤란로4길 27 2층
Metadata: {}

Document 4:
Content: 운영 요일: 매일
Metadata: {}

Document 5:
Content: 메뉴명: 쟁반짜장(2인), 메뉴 설명: 불 맛 가득! 채소와 고기 그리고 해산물을 푸짐하게 담았습니다., 가격: 16,000원
Metadata: {}

Document 6:
Content: 메뉴명: 짜장소스 1통 (350g), 메뉴 설명: 홍콩반점 수제 짜장 소스! 3통 구매시 8500원! 집에서도 간편하게 즐겨요, 가격: 3,500원
Metadata: {}

Document 7:
Content: 메뉴명: 단팥춘권(10개), 메뉴 설명: 달콤한 팥앙금이 가득! 후식으로 깔끔하게 마무리, 가격: 3,000원
Metadata: {}



/tmp/ipykernel_3609942/3339529135.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = ensemble_retriever.get_relevant_documents(input_query)


### 메타데이터

In [17]:
# Extracting information from the "Details" column to create documents
docs = []
for _, row in df.iterrows():
    details = row['Details']
    metadata = {}
    
    # Parse key-value pairs in the "Details" column
    for detail in details.split('\n'):
        if ':' in detail:
            key, value = detail.split(':', 1)
            metadata[key.strip()] = value.strip()
    
    docs.append(Document(page_content=details, metadata=metadata))

# Embeddings and vector store setup
encode_kwargs = {'normalize_embeddings': True}
model_kwargs = {'device': 'cpu'}

hf = HuggingFaceBgeEmbeddings(
    model_name=CFG.embedding_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

db = FAISS.from_documents(docs, hf)

# Save the FAISS vector store and documents as pickle
db.save_local(f"{CFG.output_path}/{store}_faiss{CFG.version}")

with open(f"{CFG.output_path}/{store}_docs{CFG.version}.pkl", "wb") as f:
    pickle.dump(docs, f)

# Load the FAISS vector store
db = FAISS.load_local(f"/mnt/data/{store}_faiss{CFG.version}", hf, allow_dangerous_deserialization=True)

# Create retrievers
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": CFG.retriever_k})
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = CFG.retriever_k

ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever],
    weights=[CFG.retriever_bert_weight, 1 - CFG.retriever_bert_weight]
)


In [20]:
# Example input and retrieving relevant documents
input_query = "영업시간이 어떻게 돼"
retrieved_docs = ensemble_retriever.get_relevant_documents(input_query)

# Print out the retrieved documents
for i, doc in enumerate(retrieved_docs, 1):
    print(f"Document {i}:\nContent: {doc.page_content}\nMetadata: {doc.metadata}\n")


Document 1:
Content: 전화번호: 02-555-8883
Metadata: {'전화번호': '02-555-8883'}

Document 2:
Content: 주소: 서울 강남구 테헤란로4길 27 2층
Metadata: {'주소': '서울 강남구 테헤란로4길 27 2층'}

Document 3:
Content: 오시는길: 강남역 3번 출구에서 16m미터
Metadata: {'오시는길': '강남역 3번 출구에서 16m미터'}

Document 4:
Content: 영업시간: 매일11:30 - 21:3020:30 라스트오더
Metadata: {'영업시간': '매일11:30 - 21:3020:30 라스트오더'}

Document 5:
Content: 메뉴명: 쟁반짜장(2인), 메뉴특징: 불 맛 가득! 채소와 고기 그리고 해산물을 푸짐하게 담았습니다., 가격: 16,000원
Metadata: {'메뉴명': '쟁반짜장(2인), 메뉴특징: 불 맛 가득! 채소와 고기 그리고 해산물을 푸짐하게 담았습니다., 가격: 16,000원'}

Document 6:
Content: 메뉴명: 멘보샤(5개), 메뉴특징: 겉바속탱! 탱글탱글한 새우살이 가득! 매콤달콤한 칠리소스와 환상의 짝꿍입니다, 가격: 9,900원
Metadata: {'메뉴명': '멘보샤(5개), 메뉴특징: 겉바속탱! 탱글탱글한 새우살이 가득! 매콤달콤한 칠리소스와 환상의 짝꿍입니다, 가격: 9,900원'}

Document 7:
Content: 메뉴명: 고추짬뽕, 메뉴특징: 내가 찾던 매운맛! 싱싱한 청양고추로 깔끔한 매운맛을 더합니다., 가격: 8,900원
Metadata: {'메뉴명': '고추짬뽕, 메뉴특징: 내가 찾던 매운맛! 싱싱한 청양고추로 깔끔한 매운맛을 더합니다., 가격: 8,900원'}

